In [8]:
import numpy as np
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize, visualize_camera_gif, visualize_projections
from datatypes.virtualCamera import VirtualCamera

In [2]:
def calculate_camera_position_WRF(focal_length, width_of_sensor, largest_radius, path_centre, camera_height):
    angle_of_view = 2*np.arctan(width_of_sensor/(2*focal_length)) # In radians
    d = largest_radius/np.tan(angle_of_view/2)
    # Assume camera parallel to x axis and always pointing towards the centre of the path circle
    x = d+path_centre[0]
    y = path_centre[1]
    z = camera_height
    return np.array([x,y,z])

def calculate_camera_pitch(camera_position):
    alpha = np.arctan(camera_position[2]/camera_position[0]) # arctan(height/distance)
    return alpha

In [3]:
def create_and_place_simple_legacy_camera(largest_radius, path_centre): # This should maybe get a better name, and should be moved somewhere?
    '''
    Function for placing a Simple legacy photo camera in the dynamic scene
    '''
    # NB! Make sure everything is in meters
    focal_length = 50*10**-3
    image_bounds = (3600, 2400) # Pixels (x,y)
    film_size = (36*10**-3, 24*10**-3)
    px = film_size[0]/image_bounds[0]
    py = film_size[1]/image_bounds[1]
    principal_point = (image_bounds[0]/2,image_bounds[1]/2)
    width_of_sensor = 36*10**-3 # Width of sensor
    camera_height = 0 # metre

    position_WRF = calculate_camera_position_WRF(focal_length, width_of_sensor, largest_radius, path_centre, camera_height)
    roll = 0
    yaw = -np.pi
    pitch = -calculate_camera_pitch(position_WRF)

    camera = VirtualCamera(position_WRF, roll, yaw, pitch, focal_length, px, py, principal_point, image_bounds)

    return camera

In [4]:
def project_all_points(camera, vessels):
    all_projected_points = {}
    for t in vessels[0].get_track().get_time_stamps():
        points = [vessel.calculate_3D_cornerpoints(t) for vessel in vessels]
        projected_points = [camera.project_points(vessel_points) for vessel_points in points]
        all_projected_points[t] = projected_points
    return all_projected_points

In [5]:
# Generate dynamic scene with random tracks
dsg = DynamicSceneGenerator()
dsg.set_random_vessels(6)
dsg.generate_random_tracks()
vessels = dsg.get_vessels()
visualize(vessels)

In [6]:
camera = create_and_place_simple_legacy_camera(dsg.get_larges_radius(), dsg.get_path_centre())
visualize_camera_gif(camera, vessels)

In [9]:
all_projected_points = project_all_points(camera, vessels)
visualize_projections(all_projected_points, camera.image_bounds)

![alt text](gifs/dynamicSceneExample.gif "Dynamic scene")

![alt text](gifs/camera_position.gif "Camera scene")


![alt text](gifs/projected_points.gif "Projected scene")
